# In this Notebook:- 
* We fine-tuned the IndicTrans2-en-indic-dist-200M model on the PHINC (Parallel Hinglish Social Media Code-Mixed Corpus for Machine Translation) for the EN→HINGLISH translation task. The resulting model was evaluated using a comprehensive suite of MT quality metrics, including BLEU, ChrF, COMET, BERTScore, and BLEURT.

[](http://)

In [11]:
pip uninstall -y numpy jax jaxlib


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
os.environ["TRANSFORMERS_NO_JAX"] = "1"


In [13]:
!pip uninstall -y numpy scipy pandas pyarrow datasets transformers
!pip install --force-reinstall --no-cache-dir \
  numpy==1.26.4 \
  scipy==1.11.4 \
  pandas==2.1.4 \
  pyarrow==14.0.2 \
  datasets==2.16.1 \
  transformers==4.36.2


Found existing installation: scipy 1.11.4
Uninstalling scipy-1.11.4:
  Successfully uninstalled scipy-1.11.4
Found existing installation: pandas 2.1.4
Uninstalling pandas-2.1.4:
  Successfully uninstalled pandas-2.1.4
Found existing installation: pyarrow 14.0.2
Uninstalling pyarrow-14.0.2:
  Successfully uninstalled pyarrow-14.0.2
Found existing installation: datasets 2.16.1
Uninstalling datasets-2.16.1:
  Successfully uninstalled datasets-2.16.1
Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 274.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 191.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 303.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 273.6 MB/s eta 0:00:00
     ━━━━━━━━━

In [14]:
#Checking wheather GPU is working or not
!nvidia-smi


Fri Dec 19 16:30:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              8W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
# installing dataset and transformer
!pip install datasets transformers[sentencepiece] sacrebleu -q

In [16]:
# to remove version conflict of Protobuf so, downgrade version of Protobuf
!pip install protobuf==3.20.3 

In [17]:
# Importing all required modules
import os
import sys
import transformers
import torch  # pytorch Import
import sacrebleu
from torch.amp import autocast, GradScaler
from tqdm.auto import tqdm
from transformers import DataCollatorForSeq2Seq
from torch.utils.data import DataLoader
from torch.optim import AdamW
from datasets import load_dataset # for loading the dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM # For getting Embedding
from transformers import DataCollatorForSeq2Seq #getting sequential model and collator for loading batchwise of data
from torch.optim import AdamW # Optimizer



In [18]:
# Enter Access Token and rerun
from huggingface_hub import login
login(new_session=False)

# Note:
* I was using the free version of Kaggle, and the memory limit was getting exhausted while training the 1B-parameter model. Because of this constraint, I switched to using the 200M-parameter model instead.

In [19]:
pip install -U transformers==4.46.3


  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.20.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)
Using cached tokenizers-0.20.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.2
    Uninstalling transformers-4.36.2:
      Successfully uninstalled transformers-4.36.2
Note: you may need to restart the kernel to use updated packages.


In [21]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL = "ai4bharat/indictrans2-en-indic-1B"
LORA_REPO  = "Vir123-dev/indictrans2_en_hing_finetune_1B"

# Load tokenizer ONLY from base model
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True
)

# Load base model
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True
)

# Attach LoRA adapters
model = PeftModel.from_pretrained(
    base_model,
    LORA_REPO
)

model = model.to("cuda")
model.eval()


RuntimeError: Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
Failed to import transformers.generation.configuration_utils because of the following error (look up to see its traceback):
cannot import name 'is_hqq_available' from 'transformers.utils' (/usr/local/lib/python3.12/dist-packages/transformers/utils/__init__.py)

# The Dataset¶

* Source: https://huggingface.co/datasets/LingoIITGN/PHINC

In [ ]:
raw_dataset = load_dataset("LingoIITGN/PHINC")

In [ ]:
raw_dataset

In [ ]:
def convert_to_translation(example):
    return {
        "translation": {
            "en": example["English_Translation"],
            "hing": example["Sentence"]
        }
    }

raw_dataset = raw_dataset.map(convert_to_translation)

In [ ]:
raw_dataset

In [ ]:
from datasets import DatasetDict

# total rows
total_rows = raw_dataset["train"].num_rows

# required splits
train_rows = 12000
valid_rows = 538
test_rows = 1200

# slice the dataset
train_dataset = raw_dataset["train"].select(range(0, train_rows))
valid_dataset = raw_dataset["train"].select(range(train_rows, train_rows + valid_rows))
test_dataset  = raw_dataset["train"].select(range(train_rows + valid_rows,
                                                  train_rows + valid_rows + test_rows))

# create final DatasetDict
final_dataset = DatasetDict({
    "train": train_dataset,
    "validation": valid_dataset,
    "test": test_dataset
})

final_dataset

In [ ]:
raw_dataset = final_dataset

# Observation for Statistics related to dataset

In [ ]:
# Import required libraries
import numpy as np
import math
import nltk
nltk.download("punkt")  # one-time

def add_stats(example):
    text = example["translation"]["en"]
    # guard
    if text is None: text = ""
    text = text.strip() # Removes unwanted spacing
    words = text.split()
    # sentence count (approx)
    sents = nltk.tokenize.sent_tokenize(text) if text else []
    example["num_words"] = len(words)
    example["num_chars"] = len(text)
    example["num_sentences"] = len(sents)
    return example

raw_dataset = raw_dataset.map(add_stats, batched=False)

In [ ]:
# Obtaining the Statistics:

def summary_stats(arr):
    arr = np.array(arr)
    return {
        "count": int(arr.size),
        "min": int(arr.min()) if arr.size>0 else None,
        "p1": int(np.percentile(arr, 1)) if arr.size>0 else None,
        "p10": int(np.percentile(arr, 10)) if arr.size>0 else None,
        "median": float(np.median(arr)) if arr.size>0 else None,
        "mean": float(arr.mean()) if arr.size>0 else None,
        "std": float(arr.std(ddof=0)) if arr.size>0 else None,
        "p90": int(np.percentile(arr, 90)) if arr.size>0 else None,
        "p99": int(np.percentile(arr, 99)) if arr.size>0 else None,
        "max": int(arr.max()) if arr.size>0 else None,
    }

for split in raw_dataset:
    d = raw_dataset[split]
    print(f"\n=== {split.upper()} ===")
    print("Words:", summary_stats(d["num_words"]))
    print("Chars:", summary_stats(d["num_chars"]))
    print("Sentences:", summary_stats(d["num_sentences"]))

In [ ]:
from datasets import DatasetDict

In [ ]:
# Train has min length of sentences as 0 ('min': 0) so, we Remove these row from dataset
def not_empty(example):
    text = example["translation"]["en"]
    return text is not None and len(text.strip()) > 0
    
clean_train = raw_dataset["train"].filter(not_empty)
clean_val   = raw_dataset["validation"].filter(not_empty)
clean_test  = raw_dataset["test"].filter(not_empty)

raw_dataset = DatasetDict({
    "train": clean_train,
    "validation": clean_val,
    "test": clean_test
})

In [ ]:
type(raw_dataset)

In [ ]:
# compute p99 threshold('p99': 31 so, removing other outliers(longer than 31 words))
word_lengths = np.array(raw_dataset["train"]["num_words"])
p99_threshold = int(np.percentile(word_lengths, 99))
print("Removing sentences longer than:", p99_threshold, "words")
raw_dataset["train"] = raw_dataset["train"].filter(
    lambda ex: ex["num_words"] <= p99_threshold
)


In [ ]:
# Obtaining the desired Statistics
def summary_stats(arr):
    arr = np.array(arr)
    return {
        "count": int(arr.size),
        "min": int(arr.min()) if arr.size>0 else None,
        "p1": int(np.percentile(arr, 1)) if arr.size>0 else None,
        "p10": int(np.percentile(arr, 10)) if arr.size>0 else None,
        "median": float(np.median(arr)) if arr.size>0 else None,
        "mean": float(arr.mean()) if arr.size>0 else None,
        "std": float(arr.std(ddof=0)) if arr.size>0 else None,
        "p90": int(np.percentile(arr, 90)) if arr.size>0 else None,
        "p99": int(np.percentile(arr, 99)) if arr.size>0 else None,
        "max": int(arr.max()) if arr.size>0 else None,
    }

for split in raw_dataset:
    d = raw_dataset[split]
    print(f"\n=== {split.upper()} ===")
    print("Words:", summary_stats(d["num_words"]))
    print("Chars:", summary_stats(d["num_chars"]))
    print("Sentences:", summary_stats(d["num_sentences"]))


In [ ]:
# Sample Example
raw_dataset['train'][0]

In [ ]:
from datasets import DatasetDict

# New desired sizes
N_TRAIN = 2000
N_VAL   = 150
N_TEST  = 250

# Downsample using .select()
small_train = raw_dataset["train"].select(range(N_TRAIN))
small_val   = raw_dataset["validation"].select(range(N_VAL))
small_test  = raw_dataset["test"].select(range(N_TEST))

# Create a new DatasetDict
small_dataset = DatasetDict({
    "train": small_train,
    "validation": small_val,
    "test": small_test
})

small_dataset

In [ ]:
model.eval() # Evaluation of model


In [ ]:
# Hinglish --> Hindi Translation

SRC_TAG = "eng_Latn"   # Hinglish / Roman Hindi
TGT_TAG = "hin_Deva"  # Hindi
def hinglish_to_hindi(sentences, batch_size=8, max_len=128):
    outputs = []

    device = model.device

    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]

        tagged = [
            f"{SRC_TAG} {TGT_TAG} {text}"
            for text in batch
        ]

        inputs = tokenizer(
            tagged,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_len
        ).to(device)

        with torch.no_grad():
            generated = model.generate(
                **inputs,
                max_length=max_len,
                num_beams=4,
                use_cache=False,       
                early_stopping=True
            )

        decoded = tokenizer.batch_decode(
            generated,
            skip_special_tokens=True
        )

        outputs.extend(decoded)

    return outputs


In [ ]:
def convert_split(split):
    hinglish_sentences = [ex["hing"] for ex in split["translation"]]
    hindi_sentences = hinglish_to_hindi(hinglish_sentences)

    new_translation = []
    for i in range(len(split)):
        new_translation.append({
            "en": split["translation"][i]["en"],
            "hing": split["translation"][i]["hing"],
            "hi": hindi_sentences[i]
        })

    return split.remove_columns("translation").add_column(
        "translation", new_translation
    )


In [ ]:
small_dataset["train"] = convert_split(small_dataset["train"])
small_dataset["validation"] = convert_split(small_dataset["validation"])
small_dataset["test"] = convert_split(small_dataset["test"])


In [ ]:
small_dataset["train"]["translation"][0]

In [ ]:
small_dataset

In [ ]:
raw_dataset = small_dataset  # As said in Task 2(for 2000 pair of sentence)

In [ ]:
import re

TECH_WORDS = [
    "ai", "ml", "ai/ml",
    "artificial intelligence",
    "machine learning",
    "emotional"
]

def protect_technical_words(text):
    protected = {}
    for i, word in enumerate(TECH_WORDS):
        token = f"<TECH{i}>"
        pattern = re.compile(re.escape(word), re.IGNORECASE)
        if pattern.search(text):
            protected[token] = word
            text = pattern.sub(token, text)
    return text, protected

def restore_technical_words(text, protected):
    for token, word in protected.items():
        text = text.replace(token.lower(), word)
        text = text.replace(token, word) 
    return text


In [ ]:
def iast_to_hinglish(text):
    # --- diacritics ---
    diacritics = {
        "ā": "a",
        "ī": "i",
        "ū": "u",
        "ṃ": "n",
        "ṅ": "ng",
        "ñ": "ny",
        "ṭ": "t",
        "ḍ": "d",
        "ṇ": "n",
        "ś": "sh",
        "ṣ": "sh",
        "ṛ": "r",
        "ḥ": "h",
    }
    for k, v in diacritics.items():
        text = text.replace(k, v)

    # --- Hinglish lexical fixes ---
    fixes = {
        " nama ": " naam ",
        " naama ": " naam ",
        " vartamana ": " vartaman ",
        " vartamaana ": " vartaman ",
        " men ": " mein ",
        " hu~": " hoon",
        " hun": " hoon",
        " aura ": " aur ",
        " yaha ": " yeh ",
        " bahuta ": " bahut ",
        " anubhava ": " anubhav ",
        "|": ".",
    }
    for k, v in fixes.items():
        text = text.replace(k, v)

    return text.lower().strip()


In [ ]:
!pip install indic-transliteration

In [ ]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

def hindi_to_hinglish(hindi_text):
    # protect technical terms
    safe_text, protected = protect_technical_words(hindi_text)

    # Hindi → IAST
    iast = transliterate(safe_text, sanscript.DEVANAGARI, sanscript.IAST)

    # IAST → Hinglish
    hinglish = iast_to_hinglish(iast)

    # restore technical terms
    hinglish = restore_technical_words(hinglish, protected)

    return hinglish


In [ ]:
Hindi_text = "@someUSER आपको ब्रिटिश बाल गायक सोफिया ग्रेस और रोजी की अपने शो की यात्रा की पहली वर्षगांठ मनाने के लिए बधाई ।"

In [ ]:
print(hindi_to_hinglish(Hindi_text))

# Evaluation metrics

# Finding  BLEU AND CHRF:

1. BLEU: BLEU checks how many n-grams from the candidate sentence also appear in the reference sentence.
2. CHRF: Instead of words, CHRF compares character n-grams.

In [ ]:
model.device

In [ ]:
len(raw_dataset["test"])

In [ ]:
n_samples = 10

# determine model device safely
try:
    model_device = next(model.parameters()).device
except StopIteration:
    model_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Model device:", model_device)

preds = []
refs = []

for i in range(min(n_samples, len(raw_dataset["test"]))):
    row = raw_dataset["test"][i]

    # handle multiple possible row formats
    if isinstance(row, dict) and "translation" in row:
        trans = row["translation"]
        # trans might be a dict or a string; handle both
        if isinstance(trans, dict):
            eng = trans.get("en") or trans.get("eng") or ""
            hi_ref = trans.get("hing") or trans.get("hin") or ""
        else:
            # sometimes translation is a string (rare) — treat as source
            eng = str(trans)
            hi_ref = ""
    elif isinstance(row, dict):
        # maybe keys are directly 'en' and 'hi'
        eng = row.get("en") or row.get("eng") or row.get("source") or ""
        hi_ref = row.get("hing") or row.get("hin") or row.get("target") or ""
    else:
        # fallback: row itself might be the translation dict-like
        try:
            eng = row["en"]
            hi_ref = row["hing"]
        except Exception:
            # last resort: stringify
            eng = str(row)
            hi_ref = ""

    eng = (eng or "").strip()
    hi_ref = (hi_ref or "").strip()
    refs.append(hi_ref if hi_ref else "")  # keep alignment
    safe_text, protected = protect_tokens(eng)
    # add required tags
    tagged = f"{SRC_TAG} {TGT_TAG} {eng}"

    # tokenize -> torch tensors -> move to model device
    tokenized = tokenizer(tagged, return_tensors="pt", padding=True, truncation=True, max_length=128)
    tokenized = {k: v.to(model_device) for k, v in tokenized.items()}

    # generate
    with torch.no_grad():
        out_ids = model.generate(
           **tokenized,
            max_length=128,
            num_beams=4,        
            use_cache=False,  
            early_stopping=True
        )
    
    Hindi_text = en_to_hi(eng, model, tokenizer)    

    hinglish = hindi_to_hinglish(Hindi_text)
    preds.append(hinglish)




In [ ]:
preds[0:4]

In [ ]:
len(preds)


In [ ]:
#BLEU:
bleu = sacrebleu.corpus_bleu(preds, [refs])
#CHRF:
chrf = sacrebleu.corpus_chrf(preds, [refs])

print(f"\nEvaluated {len(preds)} samples")
print("BLEU:", bleu.score)
print("CHRF:", chrf.score)

for i in range(min(5, len(preds))):
    print(f"\n=== SAMPLE {i+1} ===")
    print("SRC :", (raw_dataset["test"][i].get("translation", raw_dataset["test"][i]).get("en")
                    if isinstance(raw_dataset["test"][i], dict) and "translation" in raw_dataset["test"][i]
                    else (raw_dataset["test"][i].get("en") if isinstance(raw_dataset["test"][i], dict) else str(raw_dataset["test"][i]))))
    print("PRED:", preds[i])
    print("REF :", refs[i])


# Finding BERTScore:

* BERTScore: Uses BERT (or RoBERTa, or mBERT) embeddings to compare every token in candidate with every token in reference.

In [ ]:
!pip install bert-score
from bert_score import score

P, R, F1 = score(preds, refs, lang="eng")
print("BERTScore F1:", F1.mean().item())

# Finding BLEURT:
* BLEURT computes a similarity score using a fine-tuned BERT model that predicts human judgment of translation quality.

In [ ]:
# Required Packages for Bleurt
!pip install evaluate
!pip install git+https://github.com/google-research/bleurt.git

In [ ]:
# Calculating Bleurt
import evaluate
bleurt = evaluate.load("bleurt")
results = bleurt.compute(predictions=preds, references=refs)
print(results)

# Finding COMET
* COMET predicts a score that strongly correlates with human judgment.

In [ ]:
# Required package for Comet
!pip install -q unbabel-comet
from comet import download_model, load_from_checkpoint


# choose model variable 
translation_model = globals().get("translation_model", None) or globals().get("model", None)
if translation_model is None:
    raise ValueError("No translation model found. Load your model into `model` or `translation_model` first.")

# device: try to get model device (handles DeviceMap too)
try:
    model_device = next(translation_model.parameters()).device
except StopIteration:
    model_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SRC_TAG = "eng_Latn"
TGT_TAG = "hin_Deva"

srcs = []
preds = []
refs = []

n = 10
for i in range(min(n, len(raw_dataset["test"]))):
    row = raw_dataset["test"][i]

    # robust extraction of source & reference
    if isinstance(row, dict) and "translation" in row:
        trans = row["translation"]
        if isinstance(trans, dict):
            src = (trans.get("en") or trans.get("eng") or "").strip()
            ref = (trans.get("hing") or trans.get("hing") or "").strip()
        else:
            src = str(trans).strip()
            ref = ""
    elif isinstance(row, dict):
        src = (row.get("en") or row.get("eng") or row.get("source") or "").strip()
        ref = (row.get("hing") or row.get("hing") or row.get("target") or "").strip()
    else:
        # fallback
        src = str(row).strip()
        ref = ""

    srcs.append(src)
    refs.append(ref)

    # add language tags required by IndicTrans2
    tagged = f"{SRC_TAG} {TGT_TAG} {src}"

    # tokenize -> PyTorch tensors -> move to model device
    tokenized = tokenizer(tagged,
                          return_tensors="pt",
                          truncation=True,
                          padding=True,
                          max_length=128)
    tokenized = {k: v.to(model_device) for k, v in tokenized.items()}

    # generate
    with torch.no_grad():
        out_ids = translation_model.generate(**tokenized, max_length=128, num_beams=4, early_stopping=True)

    Hindi_text = en_to_hi(eng, model, tokenizer)    

    hinglish = hindi_to_hinglish(Hindi_text)
    preds.append(hinglish)

# ---- COMET evaluation ----
model_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(model_path)

# prepare data list for COMET
data = [{"src": s, "mt": p, "ref": r} for s, p, r in zip(srcs, preds, refs)]
# note: comet_model.predict returns a dict; 'scores' contains numeric values
comet_out = comet_model.predict(data, batch_size=8)
comet_scores = comet_out["scores"] if isinstance(comet_out, dict) and "scores" in comet_out else comet_out

print("Samples evaluated:", len(preds))
print("Mean COMET score:", float(sum(comet_scores) / len(comet_scores)))

# preview
for i in range(5):
    print(f"\n--- SAMPLE {i+1} ---")
    print("SRC :", srcs[i])
    print("PRED:", preds[i])
    print("REF :", refs[i])


In [ ]:
# Saving the Model and tokenizer
model.save_pretrained("pt_model")
tokenizer.save_pretrained("tokenizer")